# Récupération de tweets selon des mot en tendances

## Installation des dépendances

Installation de la librairie tweepy permettant de dialoguer avec l'api officiel de X plus intuitivement.

In [ ]:
!pip install tweepy

## Importation des bibliothèques

In [14]:
import tweepy
import pandas as pd

## Initialisation de la connexion avec l'api de X

In [15]:
bearer_token = "Le_bearer_token_fourni_par_X"

client = tweepy.Client(bearer_token)

## Récupération de tweets selon un mot en tendances renseignées

Actuellement, avec la version Basic de l'API X, nous sommes limités à un maximum de 100 tweets par mois et un minimum de 10 tweets imposés par la méthode utilisée. De plus, nous exploitons un endpoint qui ne retourne que les tweets des sept derniers jours. Enfin, nous sommes contraints à un maximum de 450 requêtes toutes les 15 minutes

In [16]:
# Mot-clé en tendance à chercher (on peut cibler un type de langue précis)
keyword = "Arcane lang:en"

# Champs des tweets, utilisateurs et lieux à récupérer
tweet_fields = [
    "id", "text", "created_at", "attachments", "author_id", "in_reply_to_user_id",
    "lang", "possibly_sensitive", "context_annotations", "public_metrics", "referenced_tweets"
]
user_fields = [
    "id", "name", "username", "description", "location", "public_metrics", "verified"
]
place_fields = ["id", "full_name", "country"]

response = client.search_recent_tweets(
    query=keyword,
    max_results=10,  # Nombre de tweets (limité par l'accès Basic)
    tweet_fields=tweet_fields,
    user_fields=user_fields,
    place_fields=place_fields,
    expansions=["author_id", "geo.place_id"]  # Récupérer les informations sur les utilisateurs et lieux
)

## Affichage des tweets dans un dataframe

In [ ]:
if response.data:
    tweets_data = []
    users_data = {user["id"]: user for user in response.includes.get("users", [])}
    places_data = {place["id"]: place for place in response.includes.get("places", [])}

    for tweet in response.data:
        tweet_info = {
            "tweet_id": tweet.id,
            "text": tweet.text,
            "created_at": tweet.created_at,
            "author_id": tweet.author_id,
            "lang": tweet.lang,
            "possibly_sensitive": tweet.possibly_sensitive,
            "retweets": tweet.public_metrics["retweet_count"],
            "replies": tweet.public_metrics["reply_count"],
            "likes": tweet.public_metrics["like_count"],
            "quotes": tweet.public_metrics["quote_count"],
        }

        # Ajouter les informations sur l'auteur si disponibles
        if tweet.author_id in users_data:
            user = users_data[tweet.author_id]
            tweet_info.update({
                "user_name": user.name,
                "username": user.username,
                "user_description": user.description,
                "user_location": user.location,
                "followers_count": user.public_metrics["followers_count"],
                "following_count": user.public_metrics["following_count"],
                "tweet_count": user.public_metrics["tweet_count"],
                "verified": user.verified,
            })

        # Ajouter les informations sur le lieu si disponibles
        if tweet.geo and tweet.geo.get("place_id") in places_data:
            place = places_data[tweet.geo["place_id"]]
            tweet_info.update({
                "place_name": place.full_name,
                "place_country": place.country,
            })

        tweets_data.append(tweet_info)

    # Créer un DataFrame
    df = pd.DataFrame(tweets_data)
    print(df)
else:
    print("Aucun tweet trouvé.")